In [124]:
#import numpy as np
import pandas as pd
import pandas_bokeh;

In [125]:
# Embedding plots in Jupyter/Colab Notebook
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [126]:
# for exporting plots as HTML
pandas_bokeh.output_file(filename='dashboard.html',title='Birds Dashboard')

In [127]:
sheet_id = '1dENiPTJtdqOiU9Oiwxen2UU2K4HP1jOR_MyGcjQP81k'
sheet_name = 'registro'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

In [128]:
df = pd.read_csv(url, usecols=[0,1,2,3,4])
#df['Costo($)'] = df['Costo($)'].replace(np.nan, 0)

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Fecha        187 non-null    object 
 1   Hora         187 non-null    object 
 2   Descripción  187 non-null    object 
 3   Cantidad(g)  187 non-null    float64
 4   Costo($)     399 non-null    float64
dtypes: float64(2), object(3)
memory usage: 15.7+ KB


In [130]:

df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Hora'] = pd.to_datetime(df['Hora'])
df['Month'] = df['Fecha'].dt.month

In [131]:
df

,Fecha,Hora,Descripción,Cantidad(g),Costo($),Month
0,2021-03-01,2021-11-12 09:00:00,Papaya,227.0,385.9,3.0
1,2021-03-01,2021-11-12 09:00:00,Plátano,165.0,379.5,3.0
2,2021-03-02,2021-11-12 09:30:00,Plátano,164.0,377.2,3.0
3,2021-03-03,2021-11-12 08:30:00,Banano,148.0,355.2,3.0
4,2021-03-03,2021-11-12 13:00:00,Banano,124.0,297.6,3.0
...,...,...,...,...,...,...
394,NaT,NaT,NaN,NaN,0.0,NaN
395,NaT,NaT,NaN,NaN,0.0,NaN
396,NaT,NaT,NaN,NaN,0.0,NaN
397,NaT,NaT,NaN,NaN,0.0,NaN


In [132]:
by_month = df.groupby(['Month']).sum()
by_month = by_month.rename(index={
    1.0:'Jan', 2.0:'Feb', 3.0:'Mar', 4.0:'Apr', 5.0:'May', 6.0:'Jun',
    7.0:'Jun', 8.0:'Ago', 9.0:'Sep', 10.0:'Oct', 11.0:'Nov', 12.0:'Dic'})

In [133]:
# by_fruit = df.groupby(['Descripción']).sum()
# by_fruit = by_fruit.drop(['Costo($)'], axis=1)
# by_fruit.pivot(columns='Cantidad(g)',values='Descripción')

In [134]:
line_plot= by_month.plot_bokeh(kind="line", y="Cantidad(g)", color='#D01C8B', plot_data_points=True, show_figure=False)

In [135]:
bar_plot = by_month.plot_bokeh(kind="bar", y='Costo($)', color='#33C7FF' ,show_figure=False)

In [136]:
# stackedbar_plot = frutis.plot_bokeh(kind='barh', stacked=True,show_figure=True)


In [137]:
#Make Dashboard with Grid Layout: 
#pandas_bokeh.plot_grid([[line_plot], [bar_plot]], width=400)